## Vanilla Predictors
**Author:** [westny](https://github.com/westny) <br>
**Date created:** 2021-05-04 <br>
**Last modified:** 2021-05-04 <br>
**Description:** Implementation of simple classifiers using CNN or RNN encoders <br>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
class VanillaCNN(nn.Module):
    def __init__(self, n_features=4, seq_len=20, n_filters=16, dropout_prob=0.7, n_outputs=3):
        super(VanillaCNN, self).__init__()
        
        self.n_outputs = n_outputs
        self.n_filters = n_filters
        self.n_filters_h = int(n_filters/2)
        self.conv1 = nn.Conv1d(n_features, n_filters, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv1d(n_filters, n_features, kernel_size=5, stride=1, padding=2)
        self.conv3 = nn.Conv1d(n_features, n_features, kernel_size=5, stride=1, padding=2)

        self.classification = nn.Sequential(
            nn.Linear(n_features*seq_len, 128),
            nn.BatchNorm1d(128),
            nn.RReLU(0.1, 0.3),
            
            nn.Dropout(0.5),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.RReLU(0.1, 0.3),
            
            nn.Dropout(0.15),
            nn.Linear(64, 32),
            nn.RReLU(0.1, 0.3),
            
            nn.Linear(32, n_outputs),
        )
    
    def forward(self, x):
        x = x[:, :, 4:8]
        x = x.permute(0, 2, 1)
        x = self.conv3(x)

        x = F.relu(x)
        x = x.contiguous().view(x.size(0), -1)
        x = self.classification(x)
        return F.softmax(x, dim=1)

In [ ]:
class VanillaLSTM(nn.Module):
    def __init__(self, n_features=4, seq_len=20, n_hidden=64, n_layers=1, dropout_prob=0.7, n_outputs=3):
        super(VanillaLSTM, self).__init__()
        
        self.n_layers = n_layers
        self.hidden_size = n_hidden
        self.lstm = nn.LSTM(n_features, n_hidden, n_layers, batch_first=True)

        self.classification = nn.Sequential(
            nn.Linear(n_hidden, 128),
            nn.BatchNorm1d(128),
            nn.RReLU(0.1, 0.3),
            
            nn.Dropout(0.5),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.RReLU(0.1, 0.3),
            
            nn.Dropout(0.15),
            nn.Linear(64, 32),
            nn.RReLU(0.1, 0.3),
            
            nn.Linear(32, n_outputs),
        )
    
    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, device='cuda'),
                torch.zeros(self.n_layers, batch_size, self.hidden_size, device='cuda'))
    
    def forward(self, x):
        x = x[:, :, 4:8]
        hc = self.init_hidden(x.size(0))
        _, (hidden, cell) = self.lstm(x, hc)
        hidden = hidden[-1,:,:]
        x = F.relu(hidden)
        x = self.classification(x)
        return F.softmax(x, dim=1)